# Владимир Смолев
## Практика в WebGame
## Предсказание конверта в плательщика по первому дню игры

Задание:
Имея данные за первый день/два игры, необходимо предсказать вероятность того, что игрок станет плательщиком. Бонус: предсказать тип плательщика (после их стандартной классификации на китов, дельфинов, пескарей и пр.)


Импортирую библиотеки

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

Загружаю таблички.

In [2]:
practice_dau = pd.read_csv('practice_dau_and.csv', sep='|', names = ['id', 'country','number_of_pgr', 'actdate'])

In [3]:
practice_pact = pd.read_csv('practice_pact_and.csv', sep='|', names = ['id', 'actdate', 'regdate', 'clusters_ml_ended',
                                                                      'revenue', 'transactions', 'quests_end',
                                                                       'm_quests_end', 'crystals_value_spend',
                                                                       'store_enters'])

In [4]:
practice_sessions = pd.read_csv('practice_sessions_and.csv', sep='|', names = ['id', 'session_number','session_length',
                                                                              'session_start_time', 'session_end_time',
                                                                              'actdate'])

## Описание данных.
### 1. practice_dau - ежедневные сообщения с устройства игрока
id - идентификатор игрока;

сountry - страна;

number_of_pgr - количество PGR-сообщений (их шлет устройство во время игры);

actdate - дата события.

### 2. practice_pact - агрегированные данные о деятельности игрока в этот день
id - идентификатор игрока;

actdate - дата события;

regdate - дата регистрации игрока;

clusters_ml_ended - сколько кластеров (кластеры состоят из квестов) основной сюжетной линии прошел игрок;

revenue - прибыль с игрока;

transactions - количество платежей;

quests_end - сколько побочных квестов прошел игрок;

m_quests_end - сколько квестов основной сюжетной линии прошел игрок;

crystals_value_spend - сколько кристаллов (в определенной нормировке) потратил игрок;

store_enters - сколько раз игрок зашел в магазин.

### 3.  practice_sessions - данные о сессиях игрока
id - идентификатор игрока;

session_number - порядковый номер сессии игрока;

session_length - длина сессии;

session_start_time - время начала сессии,

session_end_time - время окончания сессии,

actdate - дата события.

In [5]:
print(practice_dau.shape)
practice_dau.head(5)

(8519524, 4)


,id,country,number_of_pgr,actdate
0,c7a56d9eee,FR,33,2017-01-01
1,bd4b8c265d,GB,47,2017-01-20
2,bd4b8c265d,GB,6,2017-01-27
3,bd4b8c265d,GB,11,2017-01-25
4,bd4b8c265d,GB,78,2017-02-05


In [6]:
print(practice_pact.shape)
practice_pact.head(5)

(5919245, 10)


,id,actdate,regdate,clusters_ml_ended,revenue,transactions,quests_end,m_quests_end,crystals_value_spend,store_enters
0,abee37a4e1,2017-01-01,2017-01-01,7,0.0,0,61,13,0,0
1,6fbbafd123,2017-01-01,2017-01-01,5,0.0,0,73,14,0,0
2,23c025efd0,2017-01-01,2017-01-01,7,0.0,0,91,17,0,0
3,8f4b043ead,2017-01-01,2017-01-01,2,0.0,0,14,1,0,0
4,1bdb89a697,2017-01-01,2017-01-01,2,0.0,0,11,0,0,0


In [7]:
print(practice_sessions.shape)
practice_sessions.head(5)

(32533314, 6)


,id,session_number,session_length,session_start_time,session_end_time,actdate
0,14d94a58b7,13.0,57,1485368089,1485368146,2017-01-25
1,26620badd1,4.0,132,1483666420,1483666552,2017-01-06
2,01d7674aa8,7.0,83,1486676081,1486676164,2017-02-09
3,14611edffb,3.0,211,1486387335,1486387546,2017-02-06
4,3c13bd2cec,2.0,1206,1486283865,1486285071,2017-02-05


Проверим значения в выгруженных таблицах. Если там пропущенные значения.

In [8]:
print('Всего', practice_dau.isnull().sum().sum() )
print(practice_dau.isnull().sum())

Всего 1715
id                  0
country          1715
number_of_pgr       0
actdate             0
dtype: int64


In [9]:
print('Всего', practice_pact.isnull().sum().sum() )
print(practice_pact.isnull().sum())

Всего 0
id                      0
actdate                 0
regdate                 0
clusters_ml_ended       0
revenue                 0
transactions            0
quests_end              0
m_quests_end            0
crystals_value_spend    0
store_enters            0
dtype: int64


In [10]:
print('Всего', practice_sessions.isnull().sum().sum() )
print(practice_sessions.isnull().sum())

Всего 23632945
id                           0
session_number        23632945
session_length               0
session_start_time           0
session_end_time             0
actdate                      0
dtype: int64


In [11]:
23632945/practice_sessions.shape[0]

0.7264229214398509

В двух таблицах есть пропущенные значения. (Спросить что с ними делать)
В таблице по сессиим нет номера сессии в 72%. Но так то это не важно. Можно восстановить по дате если понадобится.

Выберем данные с которыми будем работатью. 

Найдем дни активности пользователей совпадающие с регистрацией.

In [11]:
print('Кол-во активности в день регистрации',
      practice_pact[practice_pact['actdate'] == practice_pact['regdate']].shape)
print('Кол-во уникальных id',
      practice_pact[practice_pact['actdate'] == practice_pact['regdate']]['id'].unique().shape)

Кол-во активности в день регистрации (2060626, 10)
Кол-во уникальных id (2016708,)


Получается, что регистрация и активность дня совпадала для некоторых id.

Видимо эти пользователи регистрировались несколько раз. Или нескольким пользователям выдавался одинаковый id при регистрации.
Хотя обе причины немного не логичны.

In [8]:
#Список id для которых есть повторы
bad_id = practice_pact[practice_pact['actdate'] == practice_pact['regdate']]['id'].value_counts()[practice_pact[practice_pact['actdate'] == practice_pact['regdate']]['id'].value_counts() > 1].index

Уберу id которые дважды регистрировались 

In [9]:
bad_id

Index(['e5690b9c2e', '91f70befd5', '13acfeb7cf', 'c5a67eaa6d', 'b2ff2a362a',
       '99791b753e', '6e1121ea7a', 'fd8c3cb864', 'f3bfbbf73e', '69907ef195',
       ...
       '67a591136e', '97d382d105', 'bb87f6df03', 'e17973362c', '1dc60b5656',
       'c00f6da37d', '7e9dce85a5', '58a9bea153', 'ce3a246a44', '476cb19481'],
      dtype='object', length=32777)

In [10]:
def bad_column(id):
    return int(id in bad_id)

In [11]:
bad_column = practice_pact['id'].apply(bad_column)

In [12]:
practice_pact['bad_column_id'] = bad_column

In [13]:
practice_pact.drop(practice_pact[practice_pact['bad_column_id'] == 1].index, axis = 0, inplace = True)

In [14]:
print('Кол-во активности в день регистрации',
      practice_pact[practice_pact['actdate'] == practice_pact['regdate']].shape)
print('Кол-во уникальных id',
      practice_pact[practice_pact['actdate'] == practice_pact['regdate']]['id'].unique().shape)

Кол-во активности в день регистрации (1983931, 11)
Кол-во уникальных id (1983931,)


In [15]:
practice_pact.drop('bad_column_id', axis = 1, inplace = True)

Готово. 

In [16]:
practice_pact['actdate'] = pd.to_datetime(practice_pact['actdate'])
practice_pact['regdate'] = pd.to_datetime(practice_pact['regdate'])

In [17]:
practice_pact.dtypes

id                              object
actdate                 datetime64[ns]
regdate                 datetime64[ns]
clusters_ml_ended                int64
revenue                        float64
transactions                     int64
quests_end                       int64
m_quests_end                     int64
crystals_value_spend             int64
store_enters                     int64
dtype: object

In [18]:
for i in practice_pact.columns[3:]:
    print('%(id)s,min: %(min)s, max %(max)s, mean %(mean)s' %
          {'id': i, 'min': practice_pact[i].min(), 'max':practice_pact[i].max(), 'mean':practice_pact[i].mean() })

clusters_ml_ended,min: -2, max 98, mean 8.092354058248882
revenue,min: 0.0, max 184607.43, mean 99.57992233366134
transactions,min: 0, max 83, mean 0.02271998539213001
quests_end,min: 0, max 796, mean 15.160549621108371
m_quests_end,min: 0, max 141, mean 3.1873663836391857
crystals_value_spend,min: 0, max 0, mean 0.0
store_enters,min: 0, max 0, mean 0.0


Выделим все первые дни. По которым будет предсказание.

In [55]:
first_day = practice_pact[practice_pact['actdate'] == practice_pact['regdate']]
#Регистрация совпадает с днем активности => первый день в игре

In [20]:
revenue_positive_id = practice_pact.groupby('id').sum()[practice_pact.groupby('id').sum()['revenue'] > 0].index
#Там, где будет единица в целевой переменной

In [56]:
first_day = first_day[first_day['revenue'] == 0]
#Те, кто заплатил в первый день не интересны.

In [22]:
practice_sessions['actdate'] = pd.to_datetime(practice_sessions['actdate'])

In [57]:
new_data_fd = pd.merge(first_day, practice_sessions, on=['id', 'actdate'], how='inner')
#добавлю данные о сессиях

In [24]:
new_data_fd.head()

,id,actdate,regdate,clusters_ml_ended,revenue,transactions,quests_end,m_quests_end,crystals_value_spend,store_enters,session_number,session_length,session_start_time,session_end_time
0,abee37a4e1,2017-01-01,2017-01-01,7,0.0,0,61,13,0,0,3.0,293,1483263907,1483264200
1,abee37a4e1,2017-01-01,2017-01-01,7,0.0,0,61,13,0,0,2.0,367,1483263100,1483263467
2,abee37a4e1,2017-01-01,2017-01-01,7,0.0,0,61,13,0,0,1.0,1826,1483261078,1483262904
3,abee37a4e1,2017-01-01,2017-01-01,7,0.0,0,61,13,0,0,5.0,290,1483282146,1483282436
4,abee37a4e1,2017-01-01,2017-01-01,7,0.0,0,61,13,0,0,6.0,2396,1483290560,1483292956


In [58]:
new_data_fd['id'].unique().shape

(1954416,)

In [159]:
#new_data_fd.groupby('id', as_index = False).sum()[['id', 'session_length']]

In [26]:
amount_visits_fd = pd.DataFrame(data = np.array([new_data_fd['id'].value_counts().index, new_data_fd['id'].value_counts().values]).T, columns=['id', 'amount_visits'])
#Количетсво посещений в день

In [27]:
amount_visits_fd.shape[0]

1954416

In [59]:
first_day = pd.merge(first_day, amount_visits_fd, on=['id'], how='inner')
first_day = pd.merge(first_day, new_data_fd.groupby('id', as_index = False).sum()[['id', 'session_length']], on=['id'], how='inner')
#добавим к данным количество посещений и длину посещения в первый день

In [60]:
first_day.store_enters.value_counts()

0    1954416
Name: store_enters, dtype: int64

In [61]:
first_day.drop(['store_enters', 'revenue', 'regdate', 'transactions', 'crystals_value_spend'], axis = 1, inplace = True)
#они вроде нулевые

In [63]:
first_day['average_visit'] = first_day['session_length']/first_day['amount_visits']

In [33]:
practice_dau.fillna(value='NAN', inplace = True)

In [34]:
practice_dau['actdate'] = pd.to_datetime(practice_dau['actdate'])

In [64]:
first_day = pd.merge(first_day, practice_dau.groupby(['id', 'actdate'], as_index = False).sum(), on=['id', 'actdate'], how='inner')
#Добавим number_of_pgr в первый день

In [65]:
def cr_answer(id):
    return int(id in revenue_positive_id)

first_day['answer'] = first_day['id'].apply(cr_answer)

In [66]:
first_day.answer.sum()

19850

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [68]:
first_day.drop(['id', 'actdate'], axis = 1, inplace = True)

In [73]:
first_day['amount_visits'] = first_day['amount_visits'].astype('int64')
first_day['average_visit'] = first_day['average_visit'].astype('float64')

In [74]:
first_day.dtypes

clusters_ml_ended      int64
quests_end             int64
m_quests_end           int64
amount_visits          int64
session_length         int64
average_visit        float64
number_of_pgr          int64
answer                 int64
dtype: object

In [77]:
#целевая переменная 
answer = first_day['answer']
first_day.drop('answer', axis=1, inplace=True)

In [78]:
#Нормализую признаки
normalizer = preprocessing.StandardScaler()
data_real_norm_np = normalizer.fit_transform(first_day)
data_real_norm_pd = pd.DataFrame(data=data_real_norm_np)

In [98]:
#обучим
X_train, X_test, y_train, y_test = train_test_split(data_real_norm_pd, answer, test_size = 0.3, random_state = 42)

LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [99]:
print('ROC-AUC = ', metrics.roc_auc_score(y_test, LogReg.predict_proba(X_test)[:, 1]))
tn, fp, fn, tp = metrics.confusion_matrix(y_test, LogReg.predict(X_test)).ravel()
print('TP = ', tp, '\t FP = ', fp) 
print('FN =', fn, '\t TN = ', tn)

ROC-AUC =  0.825300583688
TP =  0 	 FP =  57
FN = 5992 	 TN =  578645


:(

In [100]:
from sklearn.linear_model import SGDClassifier
lr_sgd = SGDClassifier(loss="log", alpha=0.05, n_iter=200, fit_intercept=True)
lr_sgd.fit(X_train, y_train)

SGDClassifier(alpha=0.05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=200, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [101]:
print('ROC-AUC = ', metrics.roc_auc_score(y_test, lr_sgd.predict_proba(X_test)[:, 1]))
tn, fp, fn, tp = metrics.confusion_matrix(y_test, lr_sgd.predict(X_test)).ravel()
print('TP = ', tp, '\t FP = ', fp) 
print('FN =', fn, '\t TN = ', tn)

ROC-AUC =  0.83342523449
TP =  1 	 FP =  19
FN = 5991 	 TN =  578683
